# GalreDB Demo

## Setup

In [1]:
import os
from pathlib import Path

import glaredb

GCP_PROJECT = os.getenv("GCP_PROJECT")
GCP_SERVICE_ACCOUNT_KEY = Path("sa.json").read_text()
GCP_STORAGE_BUCKET = f"{GCP_PROJECT}-glaredb"

SNAPSHOT_START = "2024-06-03"
SNAPSHOT_END = "2024-06-04"

con = glaredb.connect()

## Create local dataframe and external tables

In [2]:
projects = con.execute(
    f"""
    select "SnapshotAt", "Type", "Name", "OpenIssuesCount", "StarsCount", "ForksCount", "Description", "Homepage"
    from read_bigquery('{GCP_SERVICE_ACCOUNT_KEY}', '{GCP_PROJECT}', 'glaredb', 'Projects')
    where "SnapshotAt" >= '{SNAPSHOT_START}'
      and "SnapshotAt" < '{SNAPSHOT_END}'
"""
).to_polars()

con.execute(
    f"""
    create external table package_version_to_project
    from gcs
    options (
      service_account_key = '{GCP_SERVICE_ACCOUNT_KEY}',
      bucket = '{GCP_STORAGE_BUCKET}',
      location = 'PackageVersionToProject/*'
    );
"""
)

con.execute(
    f"""
    create external table package_versions
    from bigquery
    options (
      service_account_key = '{GCP_SERVICE_ACCOUNT_KEY}',
      project_id = '{GCP_PROJECT}',
      dataset_id = 'glaredb',
      table_id = 'PackageVersions'
    );
"""
)

projects

SnapshotAt,Type,Name,OpenIssuesCount,StarsCount,ForksCount,Description,Homepage
"datetime[ns, UTC]",str,str,i64,i64,i64,str,str
2024-06-03 21:00:39.949274 UTC,"""GITLAB""","""zookatron/dotenv_cli""",0,0,0,"""CLI tool for interacting with …","""https://gitlab.com/zookatron/d…"
2024-06-03 21:00:39.949274 UTC,"""GITLAB""","""zer1t0/as-search""",0,0,0,"""Search info in Autonomous Syst…","""https://gitlab.com/Zer1t0/as-s…"
2024-06-03 21:00:39.949274 UTC,"""GITLAB""","""yannbeauxis/sinagot""",0,0,0,"""Python lightweight workflow ma…","""https://gitlab.com/YannBeauxis…"
2024-06-03 21:00:39.949274 UTC,"""GITLAB""","""vuthaihoc/sitemap""",0,0,0,"""[Laravelium](https://laraveliu…","""https://gitlab.com/vuthaihoc/S…"
2024-06-03 21:00:39.949274 UTC,"""GITLAB""","""zibu/common""",0,0,0,"""通用逻辑功能模块（包含AjaxService、clipboa…","""https://gitlab.com/zibu/common"""
…,…,…,…,…,…,…,…
2024-06-03 21:00:39.949274 UTC,"""GITHUB""","""adonese/noebs-store""",3,0,0,"""A generic database for noebs (…",null
2024-06-03 21:00:39.949274 UTC,"""GITHUB""","""argentcrusade/crud-kit""",3,0,0,"""[WIP""",null
2024-06-03 21:00:39.949274 UTC,"""GITHUB""","""arbrain/abrain""",3,0,0,"""Developing a generic framework…",null


## Join local dataframe with external tables

In [6]:
con.sql(
    f"""
    select
      p."Type",
      pv."System",
      count(distinct p."Name") as "Projects"
    from projects as p
    left join package_version_to_project as pvp
      on p."SnapshotAt" = pvp."SnapshotAt"
      and p."Type" = pvp."ProjectType"
      and p."Name" = pvp."ProjectName"
    left join package_versions as pv
      on pvp."SnapshotAt" = pv."SnapshotAt"
      and pvp."System" = pv."System"
      and pvp."Name" = pv."Name"
      and pvp."Version" = pv."Version"
    where p."SnapshotAt" >= '{SNAPSHOT_START}'
      and p."SnapshotAt" < '{SNAPSHOT_END}'
      and pv."System" in ('PYPI', 'CARGO')
    group by p."Type", pv."System"
    order by "Projects" desc
"""
).to_polars()

Type,System,Projects
str,str,i64
"""GITHUB""","""PYPI""",234089
"""GITHUB""","""CARGO""",71720
"""GITLAB""","""PYPI""",5891
"""GITLAB""","""CARGO""",2447
"""BITBUCKET""","""PYPI""",2064
"""BITBUCKET""","""CARGO""",96
